In [1]:
!pip install https://github.com/AndreMiras/pycaw/archive/develop.zip

     \ 41.8 kB 408.3 kB/s 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pycaw: filename=pycaw-20240210-py3-none-any.whl size=24537 sha256=eb17119c63f62b08c76c1d35eff09e29970f60cfacc8396f86e2cb36a5b1b12e
  Stored in directory: C:\Users\acer\AppData\Local\Temp\pip-ephem-wheel-cache-z2tnynie\wheels\85\02\91\bc9c000d9cc2df176886d36a6268026ce1dda8083db4cf0825
Successfully built pycaw



[notice] A new release of pip is available: 23.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import time
import numpy as np
import math
import HandTrackingModule as htm
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

#############
wCam, hCam = 648, 480
#############

cap = cv2.VideoCapture(1)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0


detector = htm.handDetector(detectionCon=0.7)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
#volume.GetMute()
#volume.GetMasterVolumeLevel()
#volume.GetVolumeRange()
volRange = volume.GetVolumeRange()
#volume.SetMasterVolumeLevel(-20.0, None)
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volbar = 400
volper = 0

while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        # print(lmList[4], lmList[8])
        
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2
        
        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)        
        cv2.line(img, (x1,y1), (x2,y2),(255, 0, 255), 3)
        cv2.circle(img, (cx, cy), 10, (255, 0, 255), cv2.FILLED)
        
        length = math.hypot(x2-x1, y2-y1)
        
        # Hand range 50 to 300
        # volume Range -65 to 0
        
        vol = np.interp(length, [50,300], [minVol, maxVol])
        #volume for the bar
        volbar = np.interp(length, [50,300], [400, 150])
        volper = np.interp(length, [50,300], [0, 100])

        volume.SetMasterVolumeLevel(vol, None)
        
        if length<50:
            cv2.circle(img, (cx, cy), 10, (0, 255, 0), cv2.FILLED)
            
        cv2.rectangle(img, (50,150), (85,400), (0, 255, 0), 3)
        cv2.rectangle(img, (50,int(volbar)), (85,400), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, f'{int(volper)} %', (40,450), cv2.FONT_HERSHEY_COMPLEX,
                   1, (255, 0, 0), 2)
             
    
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (30,50), cv2.FONT_HERSHEY_COMPLEX,
               1, (255, 0, 0), 2)
    cv2.imshow("Image", img)
    cv2.imshow("Img", img)
    cv2.waitKey(1)
    
    

C:\Users\acer\AppData\Roaming\Python\Python39\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 